# Host-guest complex setup and simulation using SMIRNOFF

This notebook takes a SMILES string for a guest and a 3D structure for a host, and generates an initial structure of the complex using docking. It then proceeds to solvate, parameterize the system, and then minimize and do a short simulation with OpenMM.

Please note this is intended for educational purposes and comprises a worked example, not a polished tool. The usual disclaimers apply -- don't take anything here as advice on how you should set up these types of systems; this is just an example of setting up a nontrivial system with SMIRNOFF.

## Prerequisites

Before beginning, you're going to need a license to the OpenEye toolkits (free for academics), and have these installed and working, ideally in your anaconda Python distribution. Then you'll also need the `openforcefield` toolkits installed, which you can do via `conda install -c omnia openforcefield` if you are using anaconda Python. You'll also need the `oenotebook` OpenEye Jupyter notebook library installed, such as via `pip install -i https://pypi.anaconda.org/openeye/simple openeye-oenotebook`

## Import some tools we need initially

In [ ]:
from openeye import oechem # OpenEye Python toolkits
import oenotebook as oenb
# Check license
print(oechem.OEChemIsLicensed())